# loading data

In [46]:
#加载数据资源
import numpy as np
import pandas as pd


documents_dataFrame = pd.read_csv(r'D:\论文项目\量子计算\dii\ab_extract.csv');
documents_dataFrame.head()

,NOVELTY,USE,ADVANTAGE
0,The quantum repeater system (100) comprises a ...,Quantum repeater system for quantum communicat...,The quantum repeater system generates a recove...
1,The method (100) involves obtaining (102) four...,Method for generating target Gottesman-Kitaev-...,The state of the qudit is transferred to the q...
2,The circuit comprises a first amplifier (150) ...,"Circuit, such as quantum circuit element for u...",The circuit comprises a first amplifier that h...
3,The system has a computer monitoring module fo...,Quantum computer radiating and cooling pipe co...,The system has multiple radiating modes so as ...
4,The method involves establishing a fitness fun...,Active distribution network fault locating met...,The method improves the efficiency of fault lo...


In [47]:
#将数据转换成列表形式
data=documents_dataFrame.USE.to_list()

# Tokenize words and further filtering text


Let’s tokenize each sentence into a list of words, removing characters of length less than 3 and words less frequent (<10 times) in the document altogether.

In [48]:
import gensim
from gensim.utils import simple_preprocess

# Note that simple_preprocess by default delete tokens of length less than 2
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), min_len=3, deacc=True))  # min_len min lenght of a word
                                                                                    #    deacc=True removes accent marks from tokens
data_words = list(sent_to_words(data))

#  Creating Bigram and Trigram Models

Bigrams are two words frequently occurring together in the document. Trigrams are 3 words frequently occurring.Gensim's Phrases model can build and implement the bigrams, trigrams, quadgrams and more. The two important arguments to Phrases are min_count and threshold.

min_count ignore all words and bigrams with total collected count lower than this. Bydefault it value is 5

threshold represents a threshold for forming the phrases (higher means fewer phrases). A phrase of words a and b is accepted if (cnt(a, b) - min_count) * N / (cnt(a) * cnt(b)) > threshold, where N is the total vocabulary size. Bydefault it value is 10.0

In [49]:
# Build the bigram Phrase object
bigram = gensim.models.Phrases(data_words, min_count=1, threshold=1) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=1)  

# Create a Phraser object
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

#  Remove Stopwords, Make Bigrams and Lemmatize

The phrase models are ready. Let’s define the functions to remove the stopwords, make trigrams and lemmatization and call them sequentially.

In [50]:
 # NLTK Stop words
import nltk
# nltk.download('stopwords') #for the first time
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
# stop_words.extend(['come','try','go','get','make','would','really','got']) # customerize with domain specific stopword

# print(stop_words)


In [51]:
import spacy

nlp = spacy.load("en_core_web_sm")

# Define functions for stopwords, bigrams, and lemmatization
def remove_stopwords(texts):
    return [[word for word in doc if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

#def make_trigrams(texts):
#    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB','ADV']): # exclude  
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

# Count the frequency of occurance of tokens

from collections import defaultdict 
import operator

def word_frequency(corpus=[[]]):
    """
    :param corpus: a list of lists representing tokenized documents
    :return: a dict containing the frequency of each word in the corpus
    """
    frequency = defaultdict(int)
    for doc in corpus:
        for w in doc:
            frequency[w] += 1
    return dict(sorted(frequency.items(), key=operator.itemgetter(1), reverse=True))

 Let's call the functions in order.

In [52]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb,
data_lemmatized_all = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB'])

# Filtering words based off of low frequency < 10 instances (mispellings, rare words) and removing
# high frequency words that don't provide a lot of discrimination between documents
freq = word_frequency(data_lemmatized_all)

data_lemmatized = [[token for token in doc if freq[token] > 1] for doc in data_lemmatized_all]

print(data_lemmatized)

[['system', 'quantum_communication'], ['method', 'generating', 'target', 'computation', 'quantum_communication', 'system'], ['circuit', 'quantum_circuit', 'element', 'use', 'multiplexe', 'readout_system', 'qubit', 'performing_quantum', 'processing', 'operation'], ['cool', 'control_system'], ['active', 'distribution_network', 'fault', 'use', 'computer', 'device'], ['electronic_structure', 'component', 'quantum_computer'], ['quantum_compute', 'system'], ['method', 'distribute', 'quantum_entangle', 'state', 'electronic_device', 'claim'], ['fabrication', 'device_claime'], ['key', 'method', 'detect'], ['server', 'computer_tablet', 'computer_home', 'entertainment', 'computer_network', 'mobile_computing', 'head_mounte', 'augmented_reality', 'device'], ['method', 'quantum', 'analog_digital', 'conversion', 'quantum', 'digital_signal', 'processing'], ['method', 'execute', 'fermi', 'quantum_computer'], ['method_useful', 'simulating_quantum', 'circuit', 'use', 'claim'], ['method_useful', 'processi

In [53]:
import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized



# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus)

[[(0, 1), (1, 1)], [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)], [(6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1)], [(16, 1), (17, 1)], [(15, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1)], [(23, 1), (24, 1), (25, 1)], [(1, 1), (26, 1)], [(4, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1)], [(32, 1), (33, 1)], [(4, 1), (34, 1), (35, 1)], [(20, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1)], [(4, 1), (11, 1), (44, 1), (45, 1), (46, 1), (47, 2)], [(4, 1), (25, 1), (48, 1), (49, 1)], [(6, 1), (15, 1), (27, 1), (50, 1), (51, 1)], [(11, 1), (27, 1), (30, 1), (50, 1)], [(27, 1), (50, 1), (52, 1), (53, 1)], [(4, 1), (27, 1), (29, 1), (54, 1)], [(19, 1), (32, 1), (47, 1), (50, 1), (55, 1), (56, 1)], [(57, 1), (58, 1), (59, 1), (60, 1), (61, 1)], [(27, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1)], [(1, 1), (20, 1), (25, 1), (26, 1), (68, 1), (69, 1), (70, 1)], [(20, 1), (71, 1)], [(1, 1), (4, 1), (15, 1), (26,

In [55]:
print(len(id2word))

2294


# TF-IDF



###  Term Frequency - Inverse Document Frequency it makes sure that less importance is given to most frequent words and also considers less frequent words.

Term Frequency is number of times a particular word(W) occurs in a review divided by totall number of words (Wr) in review. The term frequency value ranges from 0 to 1.

Inverse Document Frequency is calculated as log(Total Number of Docs(N) / Number of Docs which contains particular word(n)). Here Docs referred as Reviews.

TF-IDF is TF * IDF that is (W/Wr)*LOG(N/n)

Using scikit-learn's tfidfVectorizer we can get the TF-IDF.

So even here we get a TF-IDF value for every word and in some cases it may consider different meaning reviews as similar after stopwords removal. so to over come we can use BI-Gram or NGram.

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer # TF-IDF
tf_idf = TfidfVectorizer(max_features=5000)
tf_data = tf_idf.fit_transform(data)
print(tf_data[1])

  (0, 2175)	0.1716855624018377
  (0, 576)	0.20410031378391677
  (0, 1497)	0.10691584895362981
  (0, 3400)	0.17096142724544286
  (0, 3042)	0.19269214276394464
  (0, 1314)	0.39295933333818295
  (0, 2408)	0.39295933333818295
  (0, 1677)	0.39295933333818295
  (0, 1326)	0.39295933333818295
  (0, 3171)	0.2774908914634861
  (0, 1293)	0.21593045502832292
  (0, 1893)	0.0893743985195235
  (0, 540)	0.20132142281463358
  (0, 1232)	0.12507986686121655
  (0, 3157)	0.10951940627633452
  (0, 2534)	0.18337170066715783


# LDA Model Traning

##  Building the base topic model and find the number of topics

We have everything required to train the base LDA model. In addition to the corpus and dictionary, you need to provide the number of topics as well. Apart from that, alpha and eta are hyperparameters that affect sparsity of the topics. According to the Gensim docs, both defaults to 1.0/num_topics prior (we'll use default for the base model).Chunksize controls how many documents are processed at a time in the training algorithm. Increasing chunksize will speed up training, at least as long as the chunk of documents easily fit into memory.Passes controls how often we train the model on the entire corpus (set to 20). Another word for passes might be "epochs". iterations is somewhat technical, but essentially it controls how often we repeat a particular loop over each document. It is important to set the number of "passes" and "iterations" high enough.

In [13]:
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary

In [10]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=1):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model=LdaModel(corpus=corpus,
                       id2word=id2word,
                       num_topics=num_topics, 
                       random_state=100,
                       chunksize=500,
                       passes=20,
                       iterations = 400,
                       eval_every = 1,
                       alpha='symmetric',
                       eta=0.07)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [11]:
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=3, limit=10, step=1)
# Show graph
import matplotlib.pyplot as plt
limit=10; start=3; step=1;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

KeyboardInterrupt: 

In [ ]:
from pprint import pprint

# Print the Keyword in the 10 topics
lda = model_list[1]
pprint(lda.print_topics())
doc_lda = lda[corpus]

lda2 = model_list[3]
pprint(lda2.print_topics())
doc_lda2 = lda2[corpus]

lda3 = model_list[5]
pprint(lda3.print_topics())
doc_lda3 = lda3[corpus]

##  Investigate Results

In [ ]:
from pprint import pprint
 import pyLDAvis.gensim
import pickle 
import pyLDAvis

# Visualize the topics
pyLDAvis.enable_notebook()

LDAvis_prepared = pyLDAvis.gensim.prepare(model_list[1], corpus, id2word)

LDAvis_prepared

In [ ]:
 # Visualize the topics
pyLDAvis.enable_notebook()

LDAvis_prepared = pyLDAvis.gensim.prepare(model_list[3], corpus, id2word)

LDAvis_prepared

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()

LDAvis_prepared = pyLDAvis.gensim.prepare(model_list[5], corpus, id2word)

LDAvis_prepared

In [ ]:
# select and article at random from reviews
random_article_index = np.random.randint(len(reviews))

print(random_article_index)
print(reviews.iloc[random_article_index,1])

In [ ]:
 print(data_lemmatized[random_article_index])

In [ ]:
dictionary = id2word
bow = dictionary.doc2bow(data_words[random_article_index])
lda.get_document_topics(bow=bow)